In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Preparing Data

In [2]:
import torchtext
from torchtext.data import Field, LabelField, BucketIterator

# Set `batch_first=True` in the `Field`.
TEXT = Field(tokenize='spacy', tokenizer_language='en_core_web_sm', include_lengths=True, batch_first=True)
LABEL = LabelField(dtype=torch.float)

train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL, root="../assets/data")
train_data, valid_data = train_data.split()

In [3]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE, 
                 vectors="glove.6B.100d", vectors_cache="../assets/vector_cache", 
                 unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [4]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, device=device)

In [5]:
for batch in train_iterator:
    break
text, text_lens = batch.text
print(text)
print(text_lens)

tensor([[ 316,  101,   78,  ...,    1,    1,    1],
        [4711,   65,    2,  ...,    1,    1,    1],
        [  11,   19, 1540,  ...,    1,    1,    1],
        ...,
        [7477,    3,   16,  ...,    1,    1,    1],
        [  56,   15,  165,  ...,    1,    1,    1],
        [ 619,   48,   13,  ...,    1,    1,    1]], device='cuda:0')
tensor([ 232,  445,  258,  619,  679,  271,  515,  169,  158,  434,  694,  606,
          98,  150,  302,  129,  296,  298,  192,  168,  771,  221,  210,  239,
         251,  266,  182,  611,   55,  177, 1307,  182,   79,  153,  266,  388,
         100,  210,   87,  120,  142,  661,  440,  404,  336,  216,  281,  299,
         177,  258,  114,  119,  156,  166,  241,  626,  112,  209,   91,  352,
         172,  337,  138,  121], device='cuda:0')


# Build Model

In [6]:
class Classifier2D(nn.Module):
    def __init__(self, in_dim, emb_dim, n_filters, filter_sizes, out_dim, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        # (1) Original implementation: 2D convolution.
        # `nn.Conv2d(in_channels, out_channels, kernel_size=(H, W))`
        self.convs = nn.ModuleList([
            nn.Conv2d(1, n_filters, kernel_size=(fs, emb_dim)) 
            for fs in filter_sizes])

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_filters*len(filter_sizes), out_dim)

    def forward(self, text, text_lens):
        # `text_lens` would be useful when mean-pooling applied. 
        # text/mask: (batch, step)
        mask = text.eq(self.emb.padding_idx)
        # embedded: (batch, step, emb)
        embedded = self.emb(text)
        
        pooled = []
        for conv_i in self.convs:
            # Note: The kernel size may exceed the sequence length, which leads to invalid output. 
            # conved_i: (batch, channel=n_filters, step-(kernel_size-1), 1)
            conved_i = F.relu(conv_i(embedded.unsqueeze(1)))
            # mask_i: (batch, 1, step-(kernel_size-1))
            mask_i = mask[:, (conv_i.kernel_size[0]-1):].unsqueeze(1)
            # pooled_i: (batch, n_filters)
            pooled_i, _ = conved_i.squeeze(-1).masked_fill(mask_i, -np.inf).max(dim=-1)
            pooled.append(pooled_i)
            
        concated = self.dropout(torch.cat(pooled, dim=-1))
        return self.fc(concated)

In [7]:
class Classifier1D(nn.Module):
    def __init__(self, in_dim, emb_dim, n_filters, filter_sizes, out_dim, dropout, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        # (2) Regard different embedding dimensions as different channels. 
        # Thus, it reduces to an 1D convolution. 
        self.convs = nn.ModuleList([
            nn.Conv1d(emb_dim, n_filters, kernel_size=fs)
            for fs in filter_sizes])

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(n_filters*len(filter_sizes), out_dim)

    def forward(self, text, text_lens):
        # `text_lens` would be useful when mean-pooling applied. 
        # text/mask: (batch, step)
        mask = text.eq(self.emb.padding_idx)
        # embedded: (batch, step, emb)
        embedded = self.emb(text)
        
        pooled = []
        for conv_i in self.convs:
            # Note: The kernel size may exceed the sequence length, which leads to invalid output. 
            # conved_i: (batch, channel=n_filters, step-(kernel_size-1))
            conved_i = F.relu(conv_i(embedded.permute(0, 2, 1)))
            # mask_i: (batch, 1, step-(kernel_size-1))
            mask_i = mask[:, (conv_i.kernel_size[0]-1):].unsqueeze(1)
            # pooled_i: (batch, n_filters)
            pooled_i, _ = conved_i.masked_fill(mask_i, -np.inf).max(dim=-1)
            pooled.append(pooled_i)
            
        concated = self.dropout(torch.cat(pooled, dim=-1))
        return self.fc(concated)

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

IN_DIM = len(TEXT.vocab)
EMB_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3, 4, 5]
OUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

classifier2d = Classifier2D(IN_DIM, EMB_DIM, N_FILTERS, FILTER_SIZES, OUT_DIM, 
                           DROPOUT, PAD_IDX).to(device)
print(count_parameters(classifier2d))

classifier1d = Classifier1D(IN_DIM, EMB_DIM, N_FILTERS, FILTER_SIZES, OUT_DIM, 
                           DROPOUT, PAD_IDX).to(device)
print(count_parameters(classifier1d))

# The two models are identical...
classifier = classifier2d

2620801
2620801


In [9]:
# Initialize Embeddings with Pre-Trained Vectors
classifier.emb.weight.data.copy_(TEXT.vocab.vectors)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

classifier.emb.weight.data[UNK_IDX].zero_()
classifier.emb.weight.data[PAD_IDX].zero_()

print(classifier.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-0.1077,  0.1105,  0.5981, -0.5436,  0.6740,  0.1066,  0.0389,  0.3548],
        [-0.3398,  0.2094,  0.4635, -0.6479, -0.3838,  0.0380,  0.1713,  0.1598]],
       device='cuda:0', grad_fn=<SliceBackward>)


# Train Model

In [10]:
optimizer = optim.Adam(classifier.parameters())
# Binary cross entropy with logits. 
# The binary version of cross entropy loss. 
loss_func = nn.BCEWithLogitsLoss().to(device)

In [11]:
def train_epoch(classifier, iterator, optimizer, loss_func):
    classifier.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        # Forward pass
        text, text_lens = batch.text
        preds = classifier(text, text_lens).squeeze(-1)
        # Calculate loss
        loss = loss_func(preds, batch.label)
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()
        # Accumulate loss and acc
        epoch_loss += loss.item()
        epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(classifier, iterator, loss_func):
    classifier.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            text, text_lens = batch.text
            preds = classifier(text, text_lens).squeeze(-1)
            # Calculate loss
            loss = loss_func(preds, batch.label)
            # Accumulate loss and acc
            epoch_loss += loss.item()
            epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [12]:
import time
N_EPOCHS = 10
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss, train_acc = train_epoch(classifier, train_iterator, optimizer, loss_func)
    valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(classifier.state_dict(), "models/tut4-model.pt")
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 19s
	Train Loss: 0.645 | Train Acc: 61.90%
	 Val. Loss: 0.503 |  Val. Acc: 76.60%
Epoch: 02 | Epoch Time: 0m 19s
	Train Loss: 0.429 | Train Acc: 80.06%
	 Val. Loss: 0.367 |  Val. Acc: 84.15%
Epoch: 03 | Epoch Time: 0m 19s
	Train Loss: 0.302 | Train Acc: 87.47%
	 Val. Loss: 0.325 |  Val. Acc: 86.22%
Epoch: 04 | Epoch Time: 0m 19s
	Train Loss: 0.220 | Train Acc: 91.49%
	 Val. Loss: 0.319 |  Val. Acc: 86.54%
Epoch: 05 | Epoch Time: 0m 19s
	Train Loss: 0.158 | Train Acc: 94.36%
	 Val. Loss: 0.338 |  Val. Acc: 86.88%
Epoch: 06 | Epoch Time: 0m 19s
	Train Loss: 0.111 | Train Acc: 96.22%
	 Val. Loss: 0.347 |  Val. Acc: 86.96%
Epoch: 07 | Epoch Time: 0m 19s
	Train Loss: 0.075 | Train Acc: 97.60%
	 Val. Loss: 0.379 |  Val. Acc: 87.01%
Epoch: 08 | Epoch Time: 0m 19s
	Train Loss: 0.053 | Train Acc: 98.40%
	 Val. Loss: 0.405 |  Val. Acc: 87.43%
Epoch: 09 | Epoch Time: 0m 19s
	Train Loss: 0.039 | Train Acc: 98.88%
	 Val. Loss: 0.440 |  Val. Acc: 87.15%
Epoch: 10 | Epoch T

In [13]:
classifier.load_state_dict(torch.load("models/tut4-model.pt"))

valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
test_loss, test_acc = eval_epoch(classifier, test_iterator, loss_func)

print(f'Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Val. Loss: 0.319 | Val. Acc: 86.54%
Test Loss: 0.354 | Test Acc: 84.87%


# User Input

In [14]:
import spacy
nlp = spacy.load('en')

def predict_senti(classifier, sentence):
    classifier.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[tok] for tok in tokenized]
    lens = len(indexed)

    # Note: `batch_first=True`
    indexed = torch.tensor(indexed, dtype=torch.long).unsqueeze(0).to(device)
    lens = torch.tensor(lens, dtype=torch.long).unsqueeze(0).to(device)
    pred = torch.sigmoid(classifier(indexed, lens)).round().type(torch.long)
    return LABEL.vocab.itos[pred.item()]

In [15]:
predict_senti(classifier, "This is a good film.")

'pos'

In [16]:
predict_senti(classifier, "This film is terrible.")

'neg'

# Check Embeddings
* The Embeddings of `<unk>` and `<pad>` tokens
    * Because the `padding_idx` has been passed to `nn.Embedding`, so the `<pad>` embedding will remain zeros throughout training.  
    * While the `<unk>` embedding will be learned.

In [17]:
print(classifier.emb.weight[:5, :8])

tensor([[ 0.0318,  0.0098,  0.1334, -0.1834, -0.0940,  0.2032, -0.0398,  0.0587],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0456, -0.2923,  0.7188, -0.4341,  0.0287,  0.0885, -0.4476,  0.2690],
        [-0.0978,  0.0405,  0.6358, -0.5522,  0.6355,  0.1571,  0.0465,  0.3754],
        [-0.3212,  0.1455,  0.4837, -0.6748, -0.4121,  0.0476,  0.1022,  0.1544]],
       device='cuda:0', grad_fn=<SliceBackward>)
